In [6]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

显示图片的函数

In [7]:
def showImage(image):
    if image is not None:
        # 使用imshow显示图像
        cv2.imshow('Image', image)

        # 等待按键后关闭窗口，0表示无限等待，可以根据需要设置等待时间，单位为毫秒
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print('Error: Image could not be read!')

消除图片中横线的函数

In [8]:
def removeLines(image_path):
    # 读取图像
    image = cv2.imread(image_path)
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 预处理：提高对比度
    equalized = cv2.equalizeHist(gray_img)

    # 使用Canny算法检测边缘
    edges = cv2.Canny(equalized, 50, 150, apertureSize=3)
    
    # 使用Hough变换检测直线，这里使用概率Hough变换，因为它对噪声更稳健
    lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)

    if lines is not None:
        # 创建一个与原图相同尺寸的空白掩码
        mask = np.zeros_like(gray_img, dtype=np.uint8) # cv2.inpaint要求掩码图像格式为“单通道，8-bit”

        # 标记横线位置
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(mask, (x1,y1), (x2,y2), (255,255,255), 3)

        # 应用inpainting技术去除横线
        dst = cv2.inpaint(image, mask, 3, cv2.INPAINT_NS)
    
    else:
        print("No horizontal line is detected.")
        dst = image

    return dst

读取测试用的样本

In [9]:
# 获取指定目录下的所有文件或目录名
dir_path = os.path.join('..', 'pics/test_data/test/normal')
items = os.listdir(dir_path)

# 存储图片文件名的列表
image_path_list = []

# 遍历所有项，挑选出图片文件
for item in items:
    item_path = os.path.join(dir_path, item)

    # 判断是否为文件
    if os.path.isfile(item_path):
        image_path_list.append(item_path)

In [10]:
i = 1
for image in image_path_list:
    image_removal = removeLines(image)
    cv2.imwrite(f'./test_pics/output{i}.png', image_removal)
    i += 1

No horizontal line is detected.
No horizontal line is detected.
No horizontal line is detected.
No horizontal line is detected.


读取图片，转为灰度图片

In [41]:
image_path = os.path.join('..', 'pics/handwritten', 'english.PNG')
image = cv2.imread(image_path)

In [43]:
im_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

使用Canny算法检测边缘

In [44]:
edges = cv2.Canny(im_gray, 100, 200, apertureSize=3)

使用Hough变换检测直线

In [45]:
lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=80, minLineLength=50, maxLineGap=10)

In [46]:
lines

array([[[ 67,  57, 313,  57]],

       [[  3,  94,   3,   0]]], dtype=int32)

遍历检测到的直线，并将其标记为白色

In [49]:
for line in lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(image, (x1,y1), (x2,y2), (255,255,255), 3)
    